## Backtest Bollinger Bands

- **Indicators:** Bollinger Bands / ATR / Pivot Levels
- **Markets:** spot
- **Position sides:** long / short
- **Timeframe:** 30m

### Strategy long:
- **Open signal:**
    - Price crossing up the bbm or price below bbl
    - Historical price data must show higher highs
    
- **Close signal:**
    - Price crossing downw the bbm or price above bbu
    - Stop loss based on ATR
    
### Strategy short:  
- **Open signal:**
    - Price crossing down the bbm or price above bbu
    - Historical price must have lower lows
    
- **Close signal:**
    - Price crossing up the bbm or price below bbl
    - Stop loss based on ATR
    
    
### Lexicon
- bbl: Bollinger lower band
- bbm: Bollinger medium band
- bbu: Bollinger upper band
- ATR: Average true range
- Pivot levels: Reversal price points



### For better data visualization and analysis, we use Jupyter Notebook to display the different scenarios. The system is using the same code for both the backtesting and live trading.
### Additionally, the analyst does not have to duplicate the code to tweak a strategy 

In [1]:
from datetime import datetime

# Importing python custom modules
from core.Timeframe import Timeframe
from datafeed.other.YahooFinance import YahooFinance
from strategies.BollingerBand import BollingerBand


/usr/local/lib/python3.8/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
# Setting the timeframe
tf = Timeframe('30m')

yf = YahooFinance()
# Pulling the price dataframe from Yahoo finance on the Apple stock for the last month until now on the 30 minutes timeframe
pf = yf.priceframe('AAPL', tf, period= '1mo')

# Backtesting the strategy on the priceframe, assuming we have 10000 as initial capital and each transaction costs 0.2%
backtest = BollingerBand.backtest(pf, cash= 10000, commission= 0.002)
stats = backtest.run()
backtest.plot()

[*********************100%***********************]  1 of 1 completed


Row(id='1469', ...)

### We get valuable information from this simple cross-over strategy to visualize patterns and optimize the parameters.
### Even though the strategy is barely profitable, the result is better than a Buy & Hold strategy, particularly during a downtrend.

In [3]:
print(stats)

Start                     2022-04-27 09:30...
End                       2022-05-26 16:00...
Duration                     29 days 06:30:00
Exposure Time [%]                   29.616725
Equity Final [$]                    9972.8016
Equity Peak [$]                  10156.453201
Return [%]                          -0.271984
Buy & Hold Return [%]               -9.052384
Return (Ann.) [%]                   -3.071537
Volatility (Ann.) [%]                8.451413
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -1.808226
Avg. Drawdown [%]                   -0.344799
Max. Drawdown Duration        9 days 01:00:00
Avg. Drawdown Duration        1 days 08:18:00
# Trades                                   32
Win Rate [%]                           78.125
Best Trade [%]                        9.83243
Worst Trade [%]                     -3.559961
Avg. Trade [%]                    

### We will now see how the same strategy react to another period.

In [4]:
pf = yf.priceframe('AAPL', tf, date_from= datetime.strptime('2022-04-01', '%Y-%m-%d'), date_to= datetime.strptime('2022-05-01', '%Y-%m-%d'))
backtest = BollingerBand.backtest(pf, cash= 10000, commission= 0.002)
stats = backtest.run()
backtest.plot()

[*********************100%***********************]  1 of 1 completed


Row(id='2220', ...)

In [5]:
print(stats)

Start                     2022-04-01 09:30...
End                       2022-05-26 16:00...
Duration                     55 days 06:30:00
Exposure Time [%]                    47.89272
Equity Final [$]                 10145.492887
Equity Peak [$]                  10154.944099
Return [%]                           1.454929
Buy & Hold Return [%]              -16.813237
Return (Ann.) [%]                   18.926273
Volatility (Ann.) [%]                6.133204
Sharpe Ratio                          3.08587
Sortino Ratio                        8.239599
Calmar Ratio                        12.651505
Max. Drawdown [%]                    -1.49597
Avg. Drawdown [%]                    -0.25078
Max. Drawdown Duration       29 days 06:00:00
Avg. Drawdown Duration        4 days 17:48:00
# Trades                                   24
Win Rate [%]                        79.166667
Best Trade [%]                       8.857638
Worst Trade [%]                     -2.294622
Avg. Trade [%]                    

### In terms of profitability, the result is similar to the previous simulation. However, other metrics, such as the Sharpe and Sortino ratio, changed significantly.
### To conclude the strategy's viability, we must backtest it on multiple periods while avoiding overfitting.

In [6]:
stats_opt = BollingerBand.optimize(
    backtest, 
    bbands_length= range(4, 30, 2), 
    bbands_std= range(1, 5), 
    order_aggreg= range(5, 15),
)
backtest.plot()

/usr/local/lib/python3.8/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 520 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/4 [00:00<?, ?it/s]

Row(id='2971', ...)

### The system defined the best parameters for this period to get the most profitability. However, we can not be sure this will be the case in the future.

In [7]:
print(stats_opt['_strategy'])

BollingerBand(bbands_length=26,bbands_std=2,order_aggreg=11)


In [8]:
print(stats_opt)


Start                     2022-04-01 09:30...
End                       2022-05-26 16:00...
Duration                     55 days 06:30:00
Exposure Time [%]                   52.873563
Equity Final [$]                 10867.148423
Equity Peak [$]                  10867.148423
Return [%]                           8.671484
Buy & Hold Return [%]              -16.813237
Return (Ann.) [%]                  171.260891
Volatility (Ann.) [%]               45.541082
Sharpe Ratio                          3.76058
Sortino Ratio                        58.69303
Calmar Ratio                       110.228733
Max. Drawdown [%]                   -1.553686
Avg. Drawdown [%]                   -0.152644
Max. Drawdown Duration        1 days 03:30:00
Avg. Drawdown Duration        0 days 10:47:00
# Trades                                   43
Win Rate [%]                            100.0
Best Trade [%]                      13.204124
Worst Trade [%]                      0.011556
Avg. Trade [%]                    

### If we want the best profitability with the least exposure time, we can tweak the outcome expectations.

In [9]:
def optim_func(series):
    
    if series['# Trades'] < 5:
        return -1
        
    return series['Equity Final [$]'] / series['Exposure Time [%]']

stats_opt = BollingerBand.optimize(
    backtest, 
    bbands_length= range(4, 30, 2), 
    bbands_std= range(1, 5), 
    order_aggreg= range(5, 15),
    maximize= optim_func,
)
backtest.plot()
print(stats_opt['_strategy'])
print(stats_opt)

/usr/local/lib/python3.8/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 520 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/4 [00:00<?, ?it/s]

BollingerBand(bbands_length=20,bbands_std=3,order_aggreg=7)
Start                     2022-04-01 09:30...
End                       2022-05-26 16:00...
Duration                     55 days 06:30:00
Exposure Time [%]                   20.306513
Equity Final [$]                  9850.934985
Equity Peak [$]                  10053.699684
Return [%]                           -1.49065
Buy & Hold Return [%]              -16.813237
Return (Ann.) [%]                   -16.49174
Volatility (Ann.) [%]                4.359377
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -2.016817
Avg. Drawdown [%]                   -0.656617
Max. Drawdown Duration       27 days 06:30:00
Avg. Drawdown Duration        9 days 00:30:00
# Trades                                    7
Win Rate [%]                        57.142857
Best Trade [%]                       1.962698
Worst Trade [%]     

### We can also use the exact same code to get realtime signals

In [10]:
signals = BollingerBand.live(pf, market= 'spot', cash= 10000, commission= 0.002)
print(signals)

Signal(market='spot', side='open', position_side='long', size=0.15, portion=None, sl=134.3007701472551, limit=None, tp=None)
Signal(market='spot', side='close', position_side='short', size=None, portion=0.3, sl=None, limit=None, tp=None)
[Signal(market='spot', side='open', position_side='long', size=0.15, portion=None, sl=134.3007701472551, limit=None, tp=None), Signal(market='spot', side='close', position_side='short', size=None, portion=0.3, sl=None, limit=None, tp=None)]


In [11]:
pf2 = pf.get()[:171]
pf2.index.name = 'Time'
pf.set(pf2)
signals = BollingerBand.live(pf, market= 'spot', cash= 10000, commission= 0.002)
print(signals)